利用Python进行数据清洗——几个简单方法

- 本篇总结一些比较便捷实用的数据清洗技巧。

# 1. 对数据集进行筛选

- data1.xlsx 为用户基本信息表。包含了用户的各项基本信息、申请时间、放款时间等。如果只需要保留申请时间在2017年1月1日之后的用户，则操作如下：

In [3]:
import pandas as pd

In [32]:
data1 = pd.read_excel('./datas/data1.xlsx')
data1

,cus_id,cus_in,apply_time,final_time,cus_edu,cert_no
0,1,张三,2000-07-05,2000-07-15,7,3213
1,2,李四,2017-01-01,2017-01-11,7,2222
2,3,王五,2002-03-04,2002-03-14,7,6912
3,4,赵六,2016-12-31,2017-01-10,7,3822
4,5,周杰伦,2022-02-03,2022-02-13,7,4712
5,6,李宇春,2023-05-01,2023-05-11,7,9323


In [20]:
data_new = data1.loc[data1['apply_time'] >= '2017-01-01']
data_new

,cus_id,cus_in,apply_time,final_time
1,2,李四,2017-01-01,2017-01-11
4,5,周杰伦,2022-02-03,2022-02-13
5,6,李宇春,2023-05-01,2023-05-11


- datanew就是我们所需要的数据。上述操作的前提是，apply_time字段的格式必须是'yyyy-mm-dd'。这种字符串格式的日期可以直接用来比较大小。

- 假如银行提供了另一张白名单客户表data2。如果想筛选出data表里的所有白名单用户的数据，则操作如下：

In [16]:
data2 = pd.read_excel('./datas/data2.xlsx')
data2

,cus_id,cus_in
0,1,张三
1,2,李四
2,3,王五
3,4,赵六
4,12,张三丰


In [18]:
data_white = data1.loc[data1['cus_id'].isin(data2['cus_id'])] # loc里面为布尔数组
data_white

,cus_id,cus_in,apply_time,final_time
0,1,张三,2000-07-05,2000-07-15
1,2,李四,2017-01-01,2017-01-11
2,3,王五,2002-03-04,2002-03-14
3,4,赵六,2016-12-31,2017-01-10


# 2. 对某一字段进行清洗

- 数据集经常存在的一个问题是数据格式不规范。例如学历字段，用数字代指学历，但同一个学历等级可能同时存在'7'、'07'、7、' 7'这四种数据标签，如果不进行处理就无法使用。处理方式如下：

In [29]:
data1['cus_edu'] = data1['cus_edu'].apply(lambda x:str(x))
data1['cus_edu'] = data1['cus_edu'].apply(lambda x:x.replace('07','7'))

- 第一行，先将该字段的所有数据都转化为str格式。这里使用了apply和lambda方法。apply可以把括号内的函数应用于整个列。

- 第二行，使用replace方法将'07'统一替换为'7'。

- 最后，' 7'替换为'7'，方法同第二行。

# 3. 变量衍生

- 变量衍生是特征工程的重要一环。根据经验和业务逻辑，可以发掘出很多解释效果很不错的衍生变量。
- 以身份证为例，从身份证当中，至少可以衍生出省市区、年龄、生日、星座和性别这些信息。
    - 身份证前两位代表省份（网上可以查到对应关系）
    - 第7-14位代表生日
    - 倒数第二位可以推断出性别，奇数为男性，偶数为女性。
    
这里就以性别为例。

## 3.1 衍生性别字段

In [33]:
data1

,cus_id,cus_in,apply_time,final_time,cus_edu,cert_no
0,1,张三,2000-07-05,2000-07-15,7,3213
1,2,李四,2017-01-01,2017-01-11,7,2222
2,3,王五,2002-03-04,2002-03-14,7,6912
3,4,赵六,2016-12-31,2017-01-10,7,3822
4,5,周杰伦,2022-02-03,2022-02-13,7,4712
5,6,李宇春,2023-05-01,2023-05-11,7,9323


In [43]:
import numpy as np

data1['gender'] = data1['cert_no'].apply(lambda x : np.where(int(str(x)[-2])%2!=0, '男', '女'))


# 出现TypeError: ‘int’ object is not subscriptable异常的情况与解决方法
# 原因：不支持索引的对象使用索引


In [44]:
data1

,cus_id,cus_in,apply_time,final_time,cus_edu,cert_no,gender
0,1,张三,2000-07-05,2000-07-15,7,3213,男
1,2,李四,2017-01-01,2017-01-11,7,2222,女
2,3,王五,2002-03-04,2002-03-14,7,6912,男
3,4,赵六,2016-12-31,2017-01-10,7,3822,女
4,5,周杰伦,2022-02-03,2022-02-13,7,4712,男
5,6,李宇春,2023-05-01,2023-05-11,7,9323,女


看起来可能略微复杂，一步步拆解来看。
- 首先看最内层的“int(x[-2])!=0”。x[-2]使用索引的方式提取出身份证号的倒数第二位，然后使用int转化为整数，再整除2。如果余数不等于0，则表示x[-2]是奇数，反之为偶数。因此这部分作为判断条件；
- 向外一层是np.where，类似于excel里面的if，它的第一个参数代表判断条件，如果条件为true则返回参数二，如果为false则返回参数三；
- 再往外则是lambda函数；
- 最后就是apply方法，它将lambda应用到整个列。整合起来看，如果身份证倒数第二位是奇数，则返回“男”，否则返回“女”。

## 3.2 列表推导式

- 列表推导式是一个超级好用的方法，它的结构是 [a if condition else b for i in list] ，运行的结果等同于：

In [ ]:
list2 = []
for i in list1:
    if condition:
        list2.append[a]
    else:
        list2.append[b]

- 其运行速度比for循环要快很多倍，且代码更简洁。
- 例如，衍生一个字段判断身份证所在省份编号和现居省份编号是否一致，1代表一致，0代表不一致：

In [46]:
data['rsd_cons'] = [1 if i == j else 0 for i, j in zip(data['cert_plc'], data['rsd_plc'])]

## 3.3 计算日期差